# Final Implementation and Report
### Group 24
### Gavin Johnson 1217930
### Gabriel Dalimonte 1208473

The core concept of our project is to add SIMD (single instruction multiple data) instructions to P0. To briefly explain, SIMD allows the CPU to execute the same instruction on multiple data elements in parallel. This greatly speeds up operations such as vector arithmetic. In order to do this several changes need to be made. The largest update is changing architecture. MIPS isn't widely used in consumer devices so the backend was changed to ARM64, a more modern architecture with wide penetration. This was the largest undertaking of the project. In addition to this, P0.py needed to be extended to allow for these operations. This is a much more forgiving and adaptable adjustment so it was left till the end. Once the backend was working properly, the front-end was an easy update. Unfortunately, the machines available to complete this project do not run on ARM64 meaning testing the output code requires a virtual simulator. While many devices exist which use the architecture all ones available to us did not allow user code execution. To deal with this, we chose to use Unicorn (http://www.unicorn-engine.org). Included in this document is the code used to emulate the execution of our ARMv8 output. This code requires the installation of the Unicorn library, which can be found at the link provided, so it will not work in this Jupyter notebook.

ARMv8 is a popular architecture and other compilers certainly exist, but the implementation below is specifically for P0 which is a unique subset of Pascal. The widespread use of ARM in mobile devices means that the knowledge gained from this project, and the potential to expand this project is high. In addition not all compilers provide as much intrinsic front-end support for SIMD instructions, which is the main goal for this project. The compilers that exist already that support SIMD don't go as deep or as native as this project.

## P0.py Front-End
First we will discuss the front end. The changes made are quite minimal, as P0 already supports some array operations. P0 was capable of defining arrays and modifying their elements, the functionality needed for SIMD operations is that of full vector arithmetic. For example:
    
    program p;
      type S = array [0..11] of integer;
      var x: S;
      var z: S;
      begin
        x := x + 2
        z := z*x
    end

In this example, two vectors are created and the entire vector is used in a addition of 2 operation, meaning every element is 2 greater. After this there is a vector-vector multiplcation. In this example the vectors or of the same type and length (which is critically important), and each element will be multiplies with the corresponding element in the other vector. i.e. z[i] = z[i]\*x[i]

In order to add this functionality, all that needed to be changed in the front-end was type relaxation on the term() and simpleExpression() methods, and calls to new methods in the back-end. The exact changes can be seen in the code below with comments beside them.

The following code block is unchanged and sets up P0, changes will be explained in later code blocks.

In [1]:
import nbimporter
from sys import argv
import SC  #  used for SC.init, SC.sym, SC.val, SC.error
from SC import TIMES, DIV, MOD, AND, PLUS, MINUS, OR, EQ, NE, LT, GT, \
     LE, GE, PERIOD, COMMA, COLON, RPAREN, RBRAK, OF, THEN, DO, LPAREN, \
     LBRAK, NOT, BECOMES, NUMBER, IDENT, SEMICOLON, END, ELSE, IF, WHILE, \
     ARRAY, RECORD, CONST, TYPE, VAR, PROCEDURE, BEGIN, PROGRAM, EOF, \
     getSym, mark
import ST  #  used for ST.init
from ST import Var, Ref, Const, Type, Proc, StdProc, Int, Bool, Enum, \
     Record, Array, newObj, find, openScope, topScope, closeScope


# first and follow sets for recursive descent parsing

FIRSTFACTOR = {IDENT, NUMBER, LPAREN, NOT}
FOLLOWFACTOR = {TIMES, DIV, MOD, AND, OR, PLUS, MINUS, EQ, NE, LT, LE, GT, GE,
                COMMA, SEMICOLON, THEN, ELSE, RPAREN, RBRAK, DO, PERIOD, END}
FIRSTEXPRESSION = {PLUS, MINUS, IDENT, NUMBER, LPAREN, NOT}
FIRSTSTATEMENT = {IDENT, IF, WHILE, BEGIN}
FOLLOWSTATEMENT = {SEMICOLON, END, ELSE}
FIRSTTYPE = {IDENT, RECORD, ARRAY, LPAREN}
FOLLOWTYPE = {SEMICOLON}
FIRSTDECL = {CONST, TYPE, VAR, PROCEDURE}
FOLLOWDECL = {BEGIN}
FOLLOWPROCCALL = {SEMICOLON, END, ELSE}
STRONGSYMS = {CONST, TYPE, VAR, PROCEDURE, WHILE, IF, BEGIN, EOF}

from sys import stdout

indent = '  '

# parsing procedures

def selector(x):
    """
    Parses
        selector = {"." ident | "[" expression "]"}.
    Assumes x is the entry for the identifier in front of the selector;
    generates code for the selector if no error is reported
    """
    while SC.sym in {PERIOD, LBRAK}:
        if SC.sym == PERIOD:  #  x.f
            getSym()
            if SC.sym == IDENT:
                if type(x.tp) == Record:
                    for f in x.tp.fields:
                        if f.name == SC.val:
                            x = CG.genSelect(x, f); break
                    else: mark("not a field")
                    getSym()
                else: mark("not a record")
            else: mark("identifier expected")
        else:  #  x[y]
            getSym(); y = expression()
            if type(x.tp) == Array:
                if y.tp == Int:
                    if type(y) == Const and \
                       (y.val < x.tp.lower or y.val >= x.tp.lower + x.tp.length):
                        mark('index out of bounds')
                    else: x = CG.genIndex(x, y)
                else: mark('index not integer')
            else: mark('not an array')
            if SC.sym == RBRAK: getSym(); 
            else: mark("] expected")
    return x

def factor():
    """
    Parses
        factor = ident selector |
                 integer  |
                 "("  expression ")" |
                 "not" factor.
    Generates code for the factor if no error is reported
    """
    if SC.sym not in FIRSTFACTOR:
        mark("expression expected"); getSym()
        while SC.sym not in FIRSTFACTOR | STRONGSYMS | FOLLOWFACTOR:
            getSym()
    if SC.sym == IDENT:
        x = find(SC.val)
        if type(x) in {Var, Ref}: x = CG.genVar(x)
        elif type(x) == Const: x = Const(x.tp, x.val); x = CG.genConst(x)
        else: mark('expression expected')
        getSym(); x = selector(x)
    elif SC.sym == NUMBER:
        x = Const(Int, SC.val); x = CG.genConst(x); getSym()
    elif SC.sym == LPAREN:
        getSym(); x = expression()
        if SC.sym == RPAREN: getSym()
        else: mark(") expected")
    elif SC.sym == NOT:
        getSym(); x = factor()
        #if (x.tp != Bool and type(x.tp) != Array) or (type(x.tp) == Array and x.tp.base != Bool): mark('not boolean')
        if x.tp != Bool: mark('not boolean')
        elif type(x) == Const: x.val = 1 - x.val # constant folding
        #elif type(x.tp) == Array and x.tp.base == Bool:
        #    x = CG.genArrayScalarOp(NOT,x,Const(Bool, 1))
        else: x = CG.genUnaryOp(NOT, x)
    else: x = Const(None, 0)
    return x

importing Jupyter notebook from SC.ipynb
importing Jupyter notebook from ST.ipynb


## Changes in Arithmetic Operations
In order to extend P0 to handle vector operations, operations such as TIMES, DIV, MOD, ADD, and MINUS had to be altered in order to accept operands of type Array. In addition the operands do not have to match types. For example: a scalar can be multiplied with a vector ie. [1, 2, 3] \* 2 = [2, 4, 6], however the underlying types must match. Finally, boolean support for SIMD operations were disabled because of time constraints and how P0 implements them.

It should be noted that a Vector \* Vector does not result the cross product of the two vectors, but rather the pairwise operations of them. For example [1,2] \* [2,3] = [2,6]

Below these changes can be seen with comments explaining the code. Combinations of vector and scalars are accepted for operations. genArrayVectorOp() and genArrayScalarOp() are defined and explained in CGARMv8.

Parses
    term = factor {("*" | "div" | "mod" | "and") factor}.
Generates code for the term if no error is reported  

In [2]:
def term():
    x = factor()
    while SC.sym in {TIMES, DIV, MOD, AND}:
        op = SC.sym; getSym();
        if op == AND and type(x) != Const: x = CG.genUnaryOp(AND, x)
        y = factor() # x op y
        if op in {TIMES, DIV, MOD}:
            if x.tp == Int == y.tp:
                if type(x) == Const == type(y): # constant folding
                    if op == TIMES: x.val = x.val * y.val
                    elif op == DIV: x.val = x.val // y.val
                    elif op == MOD: x.val = x.val % y.val
                else: x = CG.genBinaryOp(op, x, y)
            ########### ADDED CODE ############
            # Generates immediate code for the SIMD loop if
            # one or more operands is an Array and both have base type Int
            elif type(x.tp) == Array == type(y.tp) and x.tp.base == Int == y.tp.base:
                x = CG.genArrayVectorOp(op,x,y)
            elif type(y.tp) == Array and x.tp == Int == y.tp.base:
                x = CG.genArrayScalarOp(op,x,y)
            elif type(x.tp) == Array and y.tp == Int == x.tp.base:
                x = CG.genArrayScalarOp(op,x,y)
            else:
                mark('bad type')
        elif x.tp == Bool == y.tp and op == AND:
            if type(x) == Const: # constant foldingtp
                if x.val: x = y # if x is true, take y, else x
            # While this code is disabled it generated immediate code for the SIMD loop if
            # one or more operands is an Array and both have base type Bool
            #elif type(x.tp) == Array == type(y.tp) and x.tp.base == Bool == y.tp.base:
            #    x = CG.genArrayVectorOp(op,x,y)
            #elif type(y.tp) == Array and x.tp == Bool == y.tp.base:
            #    x = CG.genArrayScalarOp(op,x,y)
            #elif type(x.tp) == Array and y.tp == Bool == x.tp.base:
            #    x = CG.genArrayScalarOp(op,x,y)
            else: x = CG.genBinaryOp(AND, x, y)
            ########### END ADDED CODE ############
        else: mark('bad type')
    return x


In a very similar fashion, the addition and subtraction operations were extended to support arrays as well. Again comments in the code explain the details of the changes.

Parses
    simpleExpression = ["+" | "-"] term {("+" | "-" | "or") term}.
Generates code for the simpleExpression if no error is reported

In [3]:
def simpleExpression():
    """
    Parses
        simpleExpression = ["+" | "-"] term {("+" | "-" | "or") term}.
    Generates code for the simpleExpression if no error is reported
    """
    if SC.sym == PLUS:
        getSym(); x = term()
    elif SC.sym == MINUS:
        getSym(); x = term()
        ########### ADDED CODE ############
        # Add support for negation to be supported on Integer arrays
        if x.tp != Int and type(x.tp) != Array: mark('bad type')
        elif type(x) == Const: x.val = - x.val # constant foldingtp
        # Generate an array negation through our SIMD implementation
        # by having each element subtract itself from 0
        elif type(x.tp) == Array and x.tp.base == Int:
            x = CG.genArrayScalarOp(MINUS,Const(Int, 0),x)
        ########### END ADDED CODE ############
        else: x = CG.genUnaryOp(MINUS, x)
    else: x = term()
    while SC.sym in {PLUS, MINUS, OR}:
        op = SC.sym; getSym()
        if op == OR and type(x) != Const: x = CG.genUnaryOp(OR, x)
        y = term() # x op y
        if op in {PLUS, MINUS}:
            if x.tp == Int == y.tp:
                if type(x) == Const == type(y): # constant folding
                    if op == PLUS: x.val = x.val + y.val
                    elif op == MINUS: x.val = x.val - y.val
                else: x = CG.genBinaryOp(op, x, y)
            ########### ADDED CODE ############
            # Generates immediate code for the SIMD loop if
            # one or more operands is an Array and both have base type Int
            elif type(x.tp) == Array == type(y.tp) and x.tp.base == Int == y.tp.base:
                x = CG.genArrayVectorOp(op,x,y)
            elif type(y.tp) == Array and x.tp == Int == y.tp.base:
                x = CG.genArrayScalarOp(op,x, y)
            elif type(x.tp) == Array and y.tp == Int == x.tp.base:
                x = CG.genArrayScalarOp(op,x,y)
        elif x.tp == Bool == y.tp and op == OR:
            if type(x) == Const: # constant folding
                if not x.val: x = y # if x is false, take y, else x
            # While this code is disabled it generated immediate code for the SIMD loop if
            # one or more operands is an Array and both have base type Bool
            #elif type(x.tp) == Array == type(y.tp) and x.tp.base == Bool == y.tp.base:
            #    x = CG.genArrayVectorOp(op,x,y)
            #elif type(y.tp) == Array and x.tp == Bool == y.tp.base:
            #    x = CG.genArrayScalarOp(op,x,y)
            #elif type(x.tp) == Array and y.tp == Bool == x.tp.base:
            #    x = CG.genArrayScalarOp(op,x,y)
            ########### END ADDED CODE ############
            else: x = CG.genBinaryOp(OR, x, y)
        else: mark('bad type')
    return x

The following code from P0 was unchanged.

In [4]:
def expression():
    """
    Parses
        expression = simpleExpression
                     {("=" | "<>" | "<" | "<=" | ">" | ">=") simpleExpression}.
    Generates code for the expression if no error is reported
    """
    x = simpleExpression()
    while SC.sym in {EQ, NE, LT, LE, GT, GE}:
        op = SC.sym
        getSym(); y = simpleExpression() # x op y
        if x.tp == Int == y.tp:
            x = CG.genRelation(op, x, y)
        else: mark('bad type')
    return x

def compoundStatement():
    """
    Parses
        compoundStatement() =
            "begin" 
            statement() {";" statement()}
            "end" 
    Generates code for the compoundStatement if no error is reported
    """
    if SC.sym == BEGIN: getSym()
    else: mark("'begin' expected")
    x = statement()
    while SC.sym == SEMICOLON or SC.sym in FIRSTSTATEMENT:
        if SC.sym == SEMICOLON: getSym()
        else: mark("; missing")
        y = statement(); x = CG.genSeq(x, y)
    if SC.sym == END: getSym()
    else: mark("'end' expected")
    return x

Some changes were made to statement's type checking to allow the array operations. In order to save the changes of an array operation (such as array \* 2) assignments need to allow for array types as long as the arrays are of the same size and base type, otherwise the assignment cannot be made.

In [5]:
def statement():
    """
    Parses
        statement =
            ident selector ":=" expression |
            ident "(" [expression
                {","  expression}] ")"  |
            compoundStatement() |
            "if"  expression
                "then" statement()
                ["else" statement()] |
            "while" expression "do" statement.
    Generates code for the statement if no error is reported
    """
    if SC.sym not in FIRSTSTATEMENT:
        mark("statement expected"); getSym()
        while SC.sym not in FIRSTSTATEMENT | STRONGSYMS | FOLLOWSTATEMENT:
            getSym()
    if SC.sym == IDENT:
        x = find(SC.val); getSym()
        x = CG.genVar(x)
        if type(x) in {Var, Ref}:
            x = selector(x)
            if SC.sym == BECOMES:
                getSym(); y = expression()
                
                ####### ADDED CODE ########
                # if there is assignment of an array to an array it is allowed
                if (type(x.tp) == type(y.tp) == Array):
                    # the assignment can only be allowed if the arrays are of same length and type
                    if (x.tp.base == y.tp.base == Int and x.tp.length == y.tp.length):
                        x = CG.genDeferredAssign(x, y)
                    # else give error
                    else: mark('incompatible assignment, arrays of different size or type')
                #### END ADDED CODE #######
                
                elif x.tp == y.tp: # and not SC.error: type(y) could be Type 
                    #if type(x) == Var: ### and type(y) in {Var, Const}: incomplete, y may be Reg
                    x = CG.genAssign(x, y)
                    #else: mark('illegal assignment')
                else: mark('incompatible assignment')
            elif SC.sym == EQ:
                mark(':= expected'); getSym(); y = expression()
            else: mark(':= expected')
        elif type(x) in {Proc, StdProc}:
            fp, i = x.par, 0  #  list of formals, count of actuals
            if SC.sym == LPAREN:
                getSym()
                if SC.sym in FIRSTEXPRESSION:
                    y = expression()
                    if i < len(fp):
                        if (type(fp[i]) == Var or type(y) == Var) and \
                           fp[i].tp == y.tp:
                            if type(x) == Proc: CG.genActualPara(y, fp[i], i)
                            i = i + 1
                        else:
                            mark('illegal parameter mode')
                    else: mark('extra parameter')
                    while SC.sym == COMMA:
                        getSym()
                        y = expression()
                        if i < len(fp):
                            ####### ADDED CODE ########
                            # Reg's acceptance is a hack to support current backend operation of
                            # the address of a parameter being loaded into a register
                            if (type(fp[i]) == Var or type(y) == Var or getattr(CG, 'Reg') is not None and type(y) == CG.Reg) and \
                               fp[i].tp == y.tp:
                                if type(x) == Proc: CG.genActualPara(y, fp[i], i)
                                i = i + 1
                                ####### END ADDED CODE ########
                            else: 
                                mark('illegal parameter mode')
                        else: mark('extra parameter')
                if SC.sym == RPAREN: getSym()
                else: mark("')' expected")
            if i < len(fp): mark('too few parameters')
            if type(x) == StdProc:
                if x.name == 'read': x = CG.genRead(y)
                elif x.name == 'write': x = CG.genWrite(y)
                elif x.name == 'writeln': x = CG.genWriteln()
            else: x = CG.genCall(x)
        else: mark("variable or procedure expected")
    elif SC.sym == BEGIN: x = compoundStatement()
    elif SC.sym == IF:
        getSym(); x = expression();
        if x.tp == Bool: x = CG.genCond(x)
        else: mark('boolean expected')
        if SC.sym == THEN: getSym()
        else: mark("'then' expected")
        y = statement()
        if SC.sym == ELSE:
            if x.tp == Bool: y = CG.genThen(x, y);
            getSym()
            z = statement();
            if x.tp == Bool: x = CG.genIfElse(x, y, z)
        else:
            if x.tp == Bool: x = CG.genIfThen(x, y)
    elif SC.sym == WHILE:
        getSym(); t = CG.genTarget(); x = expression()
        if x.tp == Bool: x = CG.genCond(x)
        else: mark('boolean expected')
        if SC.sym == DO: getSym()
        else: mark("'do' expected")
        y = statement()
        if x.tp == Bool: x = CG.genWhile(t, x, y)
    else: x = None
    return x

The following P0 code was unchanged, apart from allowing ARMv8 as a targer code generation at the end.

In [6]:
def typ():
    """
    Parses
        type = ident |
               "array" "[" expression ".." expression "]" "of" type |
               "record" typedIds {";" typedIds} "end".
    Returns a type descriptor 
    """
    if SC.sym not in FIRSTTYPE:
        getSym(); mark("type expected")
        while SC.sym not in FIRSTTYPE | STRONGSYMS | FOLLOWTYPE:
            getSym()
    if SC.sym == IDENT:
        ident = SC.val; x = find(ident); getSym()
        if type(x) == Type: x = Type(x.tp)
        else: mark('not a type'); x = Type(None)
    elif SC.sym == ARRAY:
        getSym()
        if SC.sym == LBRAK: getSym()
        else: mark("'[' expected")
        x = expression()
        if SC.sym == PERIOD: getSym()
        else: mark("'.' expected")
        if SC.sym == PERIOD: getSym()
        else: mark("'.' expected")
        y = expression()
        if SC.sym == RBRAK: getSym()
        else: mark("']' expected")
        if SC.sym == OF: getSym()
        else: mark("'of' expected")
        z = typ().tp;
        if type(x) != Const or x.val < 0:
            mark('bad lower bound'); x = Type(None)
        elif type(y) != Const or y.val < x.val:
            mark('bad upper bound'); y = Type(None)
        else: x = Type(CG.genArray(Array(z, x.val, y.val - x.val + 1)))
    elif SC.sym == RECORD:
        getSym(); openScope(); typedIds(Var)
        while SC.sym == SEMICOLON:
            getSym(); typedIds(Var)
        if SC.sym == END: getSym()
        else: mark("'end' expected")
        r = topScope(); closeScope()
        x = Type(CG.genRec(Record(r)))
    else: x = Type(None)
    return x

def typedIds(kind):
    """
    Parses
        typedIds = ident {"," ident} ":" type.
    Updates current scope of symbol table
    Assumes kind is Var or Ref and applies it to all identifiers
    Reports an error if an identifier is already defined in the current scope
    """
    if SC.sym == IDENT: tid = [SC.val]; getSym()
    else: mark("identifier expected"); tid = []
    while SC.sym == COMMA:
        getSym()
        if SC.sym == IDENT: tid.append(SC.val); getSym()
        else: mark('identifier expected')
    if SC.sym == COLON:
        getSym(); tp = typ().tp
        if tp != None:
            for i in tid: newObj(i, kind(tp))
    else: mark("':' expected")

def declarations(allocVar):
    """
    Parses
        declarations =
            {"const" ident "=" expression ";"}
            {"type" ident "=" type ";"}
            {"var" typedIds ";"}
            {"procedure" ident ["(" [["var"] typedIds {";" ["var"] typedIds}] ")"] ";"
                declarations compoundStatement ";"}.
    Updates current scope of symbol table.
    Reports an error if an identifier is already defined in the current scope.
    For each procedure, code is generated
    """
    if SC.sym not in FIRSTDECL | FOLLOWDECL:
        getSym(); mark("'begin' or declaration expected")
        while SC.sym not in FIRSTDECL | STRONGSYMS | FOLLOWDECL: getSym()
    while SC.sym == CONST:
        getSym()
        if SC.sym == IDENT:
            ident = SC.val; getSym()
            if SC.sym == EQ: getSym()
            else: mark("= expected")
            x = expression()
            if type(x) == Const: newObj(ident, x)
            else: mark('expression not constant')
        else: mark("constant name expected")
        if SC.sym == SEMICOLON: getSym()
        else: mark("; expected")
    while SC.sym == TYPE:
        getSym()
        if SC.sym == IDENT:
            ident = SC.val; getSym()
            if SC.sym == EQ: getSym()
            else: mark("= expected")
            x = typ(); newObj(ident, x)  #  x is of type ST.Type
            if SC.sym == SEMICOLON: getSym()
            else: mark("; expected")
        else: mark("type name expected")
    start = len(topScope())
    while SC.sym == VAR:
        getSym(); typedIds(Var)
        if SC.sym == SEMICOLON: getSym()
        else: mark("; expected")
    varsize = allocVar(topScope(), start)
    while SC.sym == PROCEDURE:
        getSym()
        if SC.sym == IDENT: getSym()
        else: mark("procedure name expected")
        ident = SC.val; newObj(ident, Proc([])) #  entered without parameters
        sc = topScope()
        CG.procStart(); openScope() # new scope for parameters and body
        if SC.sym == LPAREN:
            getSym()
            if SC.sym in {VAR, IDENT}:
                if SC.sym == VAR: getSym(); typedIds(Ref)
                else: typedIds(Var)
                while SC.sym == SEMICOLON:
                    getSym()
                    if SC.sym == VAR: getSym(); typedIds(Ref)
                    else: typedIds(Var)
            else: mark("formal parameters expected")
            fp = topScope()
            sc[-1].par = fp[:] #  procedure parameters updated
            if SC.sym == RPAREN: getSym()
            else: mark(") expected")
        else: fp = []
        parsize = CG.genFormalParams(fp)
        if SC.sym == SEMICOLON: getSym()
        else: mark("; expected")
        localsize = declarations(CG.genLocalVars)
        CG.genProcEntry(ident, parsize, localsize)
        x = compoundStatement(); CG.genProcExit(x, parsize, localsize)
        closeScope() #  scope for parameters and body closed
        if SC.sym == SEMICOLON: getSym()
        else: mark("; expected")
    return varsize

def program():
    """
    Parses
        program = "program" ident ";" declarations compoundStatement(1).
    Generates code if no error is reported
    """
    newObj('boolean', Type(Bool)); Bool.size = 4
    newObj('integer', Type(Int)); Int.size = 4
    newObj('true', Const(Bool, 1))
    newObj('false', Const(Bool, 0))
    newObj('read', StdProc([Ref(Int)]))
    newObj('write', StdProc([Var(Int)]))
    newObj('writeln', StdProc([]))
    CG.progStart()
    if SC.sym == PROGRAM: getSym()
    else: mark("'program' expected")
    ident = SC.val
    if SC.sym == IDENT: getSym()
    else: mark('program name expected')
    if SC.sym == SEMICOLON: getSym()
    else: mark('; expected')
    declarations(CG.genGlobalVars); CG.progEntry(ident)
    x = compoundStatement()
    return CG.progExit(x)

def compileString(src, dstfn = None, target = 'armv8'):
    """Compiles string src; if dstfn is provided, the code is written to that
    file, otherwise printed on the screen"""
    global CG
    #  used for init, genRec, genArray, progStart, genGlobalVars, \
    #  progEntry, progExit, procStart, genFormalParams, genActualPara, \
    #  genLocalVars, genProcEntry, genProcExit, genSelect, genIndex, \
    #  genVar, genConst, genUnaryOp, genBinaryOp, genRelation, genSeq, \
    #  genAssign, genCall, genRead, genWrite, genWriteln, genCond, \
    #  genIfThen, genThen, genIfElse, genTarget, genWhile
    if target == 'mips': import CGmips as CG
    elif target == 'ast': import CGast as CG
    elif target == 'armv8': import CGARMv8 as CG #### added for armv8 support.
    else: print('unknown target'); return
    SC.init(src)
    ST.init()
    CG.init()
    p = program()
    if p != None and not SC.error:
        if dstfn == None: print(p)
        else:
            with open(dstfn, 'w') as f: f.write(p);

def compileFile(srcfn):
    if srcfn.endswith('.p'):
        with open(srcfn, 'r') as f: src = f.read()
        dstfn = srcfn[:-2] + '.s'
        compileString(src, dstfn)
    else: print("'.p' file extension expected")

# sampe usage:
# import os
# os.chdir('/path/to/my/directory')
# compileFile('myprogram.p')

Finally, due to the change in the code gen the default code generation file was changed to **CGARMv8** as opposed to CGmips.

In [7]:
compileString("""
program p;
  type S = array [0..14] of integer;
  var y: integer;
  var x: S;
  var z: S;
  begin
    z[0] := 1;
    z[14] := 1;
    x := (z+3) * (-z + (7 * (-(1 + z))));
    x := -x;
    x := x mod 6;
    write(x[0]); writeln(); {prints 0}
    write(x[1]); writeln(); {prints 3}
    write(x[11]); writeln(); {prints 3}
    write(x[12]); writeln(); {prints 3}
    write(x[13]); writeln(); {prints 3}
    write(x[14]); writeln() {prints 0}
    {x := x * z;}
  end
""")

importing Jupyter notebook from CGARMv8.ipynb
	.data
z_:	.space 60
x_:	.space 60
y_:	.space 4
	.text
	.global main
main:	
	adrp x14, z_
	add w14, w14, :lo12:z_
	add w14, w14, #0
	mov w11, #1
	str w11, [x14]
	adrp x12, z_
	add w12, w12, :lo12:z_
	add w12, w12, #56
	mov w9, #1
	str w9, [x12]
	mov w13, wzr
	mov w10, #12
.L0:	
	adrp x15, z_
	mov w14, #4
	madd w15, w14, w13, w15
	add w15, w15, :lo12:z_
	ld1 {v18.4S}, [x15]
	mov w11, #3
	dup v6.4S, w11
	add v27.4S, v18.4S, v6.4S
	dup v30.4S, wzr
	adrp x12, z_
	mov w9, #4
	madd w12, w9, w13, w12
	add w12, w12, :lo12:z_
	ld1 {v20.4S}, [x12]
	sub v13.4S, v30.4S, v20.4S
	mov w15, #1
	dup v0.4S, w15
	adrp x14, z_
	mov w11, #4
	madd w14, w11, w13, w14
	add w14, w14, :lo12:z_
	ld1 {v4.4S}, [x14]
	add v3.4S, v0.4S, v4.4S
	dup v15.4S, wzr
	sub v8.4S, v15.4S, v3.4S
	mov w12, #7
	dup v7.4S, w12
	mul v26.4S, v7.4S, v8.4S
	add v13.4S, v13.4S, v26.4S
	mul v27.4S, v27.4S, v13.4S
	adrp x9, x_
	mov w15, #4
	madd w9, w15, w13, w9
	add w9, w9, :lo12:x_
	st1 {v

## Backend documentation can be seen in CGARMv8.ipynb

## Unicorn CPU Emulation
### Beyond this point code does not compile correctly

In an attempt to set up a virtual testing environment with Unicorn we downloaded the libraries available at http://www.unicorn-engine.org and installed the python binding. We did set up a minimum viable environment to test our code, similar to QtSPIM. There is a necessary instruction preamble to enable SIMD and float instructions on the processor which are statically included in every program loaded. It should be noted the both unicorn code and opcode_scraper.py are designed to run in Python 2.

The unicorn code reads binary code from standard in, allowing compiled binary output to be directly piped into the emulator with a terminal command such as "hex.out | python unicorn.py"

In [ ]:
from __future__ import print_function
from unicorn import *
from unicorn.arm64_const import *
import sys

code = sys.stdin.read()

# Processor preamble to enable floating point ops and NEON (SIMD) ops
# http://infocenter.arm.com/help/topic/com.arm.doc.den0024a/BABGBFBF.html
# http://infocenter.arm.com/help/topic/com.arm.doc.den0024a/CEGDJDJD.html
NEON_ENABLE = b"\x41\x10\x38\xd5"   # mrs x1, cpacr_el1
NEON_ENABLE += b"\x21\x04\x6c\xb2"  # orr x1, x1, #0x300000
NEON_ENABLE += b"\x41\x10\x18\xd5"  # msr cpacr_el1, x1
NEON_ENABLE += b"\xdf\x3f\x03\xd5"  # isb

# code to be emulated
ARM64_CODE = NEON_ENABLE + code

# memory address where emulation starts
ADDRESS = 0x1000000

Hooks are used for simple debugging to print which instructions are being executed during each step.

In [ ]:
# callback for tracing instructions
def hook_code(uc, address, size, user_data):
    print(">>> Tracing instruction at 0x%x, instruction size = 0x%x" %(address, size))

    reg_XN = []
    reg_XN.append(mu.reg_read(UC_ARM64_REG_X9))
    reg_XN.append(mu.reg_read(UC_ARM64_REG_X10))
    reg_XN.append(mu.reg_read(UC_ARM64_REG_X11))
    reg_XN.append(mu.reg_read(UC_ARM64_REG_X12))
    reg_XN.append(mu.reg_read(UC_ARM64_REG_X13))
    reg_XN.append(mu.reg_read(UC_ARM64_REG_X14))
    reg_XN.append(mu.reg_read(UC_ARM64_REG_X15))
    reg_XN.append(mu.reg_read(UC_ARM64_REG_X16))

    j = 0
    for i in reg_XN:
        print(">>> X%d = 0x%x" %(j+9, i))
        j = j + 1

    print(">>> SP = 0x%x"%mu.reg_read(UC_ARM64_REG_SP))
    '''for i in range(0x108d010, 0x108d070, 4):
        tmp = mu.mem_read(i, 4)
        print(">>> Read 4 bytes from [0x%x] = 0x" %(i), end="")
        for i in reversed(tmp):
            print("%x" %(i), end="")
    print("")'''
    sys.stdout.flush()

The hook_intr function provides a necessary feature. It responds to our code's primitive syscalls. It doubled as a debugging facility as it would be called if the processor raised a non-syscall interrupt as well.

In [ ]:
def hook_intr(uc, exception_index, user_data):
    print("exception index: %d"%exception_index)

    # Handle SVC call
    if exception_index == 2:  # 2 is a constant number in unicorn
        syscall_op = mu.reg_read(UC_ARM64_REG_X8)
        # syscall codes and argument order and purpose taken from
        # https://w3challs.com/syscalls/?arch=arm_strong
        # It should be noted to more closely emulate the syscalls provided by the
        # original MIPS codegen that char* from the actual syscalls is the memory
        # location for which to read/write a len (x2) integer.
        if syscall_op == 93:  # Exit syscall
            exit_code = mu.reg_read(UC_ARM64_REG_X0)
            print("Program exited with status %d"%exit_code)
            uc.emu_stop()
            return
        elif syscall_op == 63:  # Read syscall
            print("Requesting integer input of size %d"%mu.reg_read(UC_ARM64_REG_X2))
            fd = mu.reg_read(UC_ARM64_REG_X0)
            if fd != 0:
                print("Warning: Non stdin fd specified (%d)"%fd)
            addr = mu.reg_read(UC_ARM64_REG_X1)
            mu.mem_write(addr, struct.pack('<i', 40))
            return
        elif syscall_op == 64:  # Write syscall
            print("Requesting integer output of size %d"%mu.reg_read(UC_ARM64_REG_X2))
            fd = mu.reg_read(UC_ARM64_REG_X0)
            if fd != 1 and fd != 2:
                print("Warning: Non stdout/stderr fd specified (%d)"%fd)
            addr = mu.reg_read(UC_ARM64_REG_X1)
            #mu.mem_write(addr, struct.pack('<i', 40))
            sz = mu.reg_read(UC_ARM64_REG_X2)
            val = mu.mem_read(addr, sz)
            # Hack
            if sz == 1 and val == '\x0A':
                print(file=sys.stderr)
            else:
                print(struct.unpack('<i',str(val))[0], end='', file=sys.stderr)
            return
        else:
            print("CPU received unhandled software exception %d"%syscall_op)

    # If an execption is still running here then it is a real problem,
    # dump the state
    reg_XN = []
    reg_XN.append(mu.reg_read(UC_ARM64_REG_X9))
    reg_XN.append(mu.reg_read(UC_ARM64_REG_X10))
    reg_XN.append(mu.reg_read(UC_ARM64_REG_X11))
    reg_XN.append(mu.reg_read(UC_ARM64_REG_X12))
    reg_XN.append(mu.reg_read(UC_ARM64_REG_X13))
    reg_XN.append(mu.reg_read(UC_ARM64_REG_X14))
    reg_XN.append(mu.reg_read(UC_ARM64_REG_X15))
    reg_XN.append(mu.reg_read(UC_ARM64_REG_X16))

    j = 0
    for i in reg_XN:
        print(">>> X%d = 0x%x" %(j+9, i))
        j = j + 1

    print(">>> W0 = 0x%x"%mu.reg_read(UC_ARM64_REG_W0))
    print(">>> W8 = 0x%x"%mu.reg_read(UC_ARM64_REG_W8))
    print(">>> W9 = 0x%x"%mu.reg_read(UC_ARM64_REG_W9))
    print(">>> W11 = 0x%x"%mu.reg_read(UC_ARM64_REG_W10))
    print(">>> W12 = 0x%x"%mu.reg_read(UC_ARM64_REG_W11))
    print(">>> W13 = 0x%x"%mu.reg_read(UC_ARM64_REG_W12))
    print(">>> W14 = 0x%x"%mu.reg_read(UC_ARM64_REG_W13))
    print(">>> W15 = 0x%x"%mu.reg_read(UC_ARM64_REG_W14))
    print(">>> W16 = 0x%x"%mu.reg_read(UC_ARM64_REG_W15))

    print(">>> V0 = 0x%x"%mu.reg_read(UC_ARM64_REG_V0))
    print(">>> V1 = 0x%x"%mu.reg_read(UC_ARM64_REG_V1))
    print(">>> V2 = 0x%x"%mu.reg_read(UC_ARM64_REG_V2))
    print(">>> V3 = 0x%x"%mu.reg_read(UC_ARM64_REG_V3))
    print(">>> V4 = 0x%x"%mu.reg_read(UC_ARM64_REG_V4))
    print(">>> V5 = 0x%x"%mu.reg_read(UC_ARM64_REG_V5))
    print(">>> V6 = 0x%x"%mu.reg_read(UC_ARM64_REG_V6))
    print(">>> V7 = 0x%x"%mu.reg_read(UC_ARM64_REG_V7))
    print(">>> V8 = 0x%x"%mu.reg_read(UC_ARM64_REG_V8))
    print(">>> V9 = 0x%x"%mu.reg_read(UC_ARM64_REG_V9))
    print(">>> V11 = 0x%x"%mu.reg_read(UC_ARM64_REG_V10))
    print(">>> V12 = 0x%x"%mu.reg_read(UC_ARM64_REG_V11))
    print(">>> V13 = 0x%x"%mu.reg_read(UC_ARM64_REG_V12))
    print(">>> V14 = 0x%x"%mu.reg_read(UC_ARM64_REG_V13))
    print(">>> V15 = 0x%x"%mu.reg_read(UC_ARM64_REG_V14))
    print(">>> V16 = 0x%x"%mu.reg_read(UC_ARM64_REG_V15))
    print(">>> V23 = 0x%x"%mu.reg_read(UC_ARM64_REG_V23))
    uc.emu_stop()

Address space is allocated for the emulation, architecture is specified and the code is ran. Registers can be checked to verify correct output.

In [10]:
print("Emulate arm code")
try:
    # Initialize emulator in ARM-64bit mode
    mu = Uc(UC_ARCH_ARM64, UC_MODE_ARM)

    # map 2MB memory for this emulation
    mu.mem_map(ADDRESS, 10 * 1024 * 1024)

    # write machine code to be emulated to memory
    mu.mem_write(ADDRESS+0x658-len(NEON_ENABLE), ARM64_CODE)  # Offset is needed to keep the relative value of data the same

    # Shoehorn stack
    mu.reg_write(UC_ARM64_REG_SP, ADDRESS + 9 * 1024 * 1024)
    mu.reg_write(UC_ARM64_REG_X29, ADDRESS + 9 * 1024 * 1024)

    # tracing all instructions with customized callback
    mu.hook_add(UC_HOOK_CODE, hook_code)

    mu.hook_add(UC_HOOK_INTR, hook_intr)

    # initialize machine registers
    #mu.reg_write(UC_ARM64_REG_X11, 0x0)
    #mu.reg_write(UC_ARM64_REG_X13, 0x10)
    #mu.reg_write(UC_ARM64_REG_X15, 0x3)
    # mu.reg_write(UC_ARM64_REG_X0, 0x7890)
    # etc...

    # emulate code in infinite time & unlimited instructions
    # The address offset is to match the offset provided by
    # objdump so all global variable have the expected address
    # Another stipulation is the first instruction from "main"
    # of the assembly is expected to be the first instruction
    # Unicorn will read.
    mu.emu_start(ADDRESS+0x658-len(NEON_ENABLE), len(ARM64_CODE))

    # now print out some registers
    print("Emulation done. Below is the CPU context")

    reg_XN = []
    reg_XN.append(mu.reg_read(UC_ARM64_REG_X9))
    reg_XN.append(mu.reg_read(UC_ARM64_REG_X10))
    reg_XN.append(mu.reg_read(UC_ARM64_REG_X11))
    reg_XN.append(mu.reg_read(UC_ARM64_REG_X12))
    reg_XN.append(mu.reg_read(UC_ARM64_REG_X13))
    reg_XN.append(mu.reg_read(UC_ARM64_REG_X14))
    reg_XN.append(mu.reg_read(UC_ARM64_REG_X15))
    reg_XN.append(mu.reg_read(UC_ARM64_REG_X16))

    j = 0
    for i in reg_XN:
        print(">>> X%d = 0x%x" %(j+9, i))
        j = j + 1

    print(">>> W9 = 0x%x"%mu.reg_read(UC_ARM64_REG_W9))
    print(">>> W11 = 0x%x"%mu.reg_read(UC_ARM64_REG_W10))
    print(">>> W12 = 0x%x"%mu.reg_read(UC_ARM64_REG_W11))
    print(">>> W13 = 0x%x"%mu.reg_read(UC_ARM64_REG_W12))
    print(">>> W14 = 0x%x"%mu.reg_read(UC_ARM64_REG_W13))
    print(">>> W15 = 0x%x"%mu.reg_read(UC_ARM64_REG_W14))
    print(">>> W16 = 0x%x"%mu.reg_read(UC_ARM64_REG_W15))

    print(">>> V0 = 0x%x"%mu.reg_read(UC_ARM64_REG_V0))
    print(">>> V1 = 0x%x"%mu.reg_read(UC_ARM64_REG_V1))
    print(">>> V2 = 0x%x"%mu.reg_read(UC_ARM64_REG_V2))
    print(">>> V3 = 0x%x"%mu.reg_read(UC_ARM64_REG_V3))
    print(">>> V4 = 0x%x"%mu.reg_read(UC_ARM64_REG_V4))
    print(">>> V5 = 0x%x"%mu.reg_read(UC_ARM64_REG_V5))
    print(">>> V6 = 0x%x"%mu.reg_read(UC_ARM64_REG_V6))
    print(">>> V7 = 0x%x"%mu.reg_read(UC_ARM64_REG_V7))
    print(">>> V8 = 0x%x"%mu.reg_read(UC_ARM64_REG_V8))
    print(">>> V9 = 0x%x"%mu.reg_read(UC_ARM64_REG_V9))
    print(">>> V11 = 0x%x"%mu.reg_read(UC_ARM64_REG_V10))
    print(">>> V12 = 0x%x"%mu.reg_read(UC_ARM64_REG_V11))
    print(">>> V13 = 0x%x"%mu.reg_read(UC_ARM64_REG_V12))
    print(">>> V14 = 0x%x"%mu.reg_read(UC_ARM64_REG_V13))
    print(">>> V15 = 0x%x"%mu.reg_read(UC_ARM64_REG_V14))
    print(">>> V16 = 0x%x"%mu.reg_read(UC_ARM64_REG_V15))
    print(">>> V21 = 0x%x"%mu.reg_read(UC_ARM64_REG_V21))
    print(">>> V23 = 0x%x"%mu.reg_read(UC_ARM64_REG_V23))
    print(">>> V24 = 0x%x"%mu.reg_read(UC_ARM64_REG_V24))
    print(">>> V25 = 0x%x"%mu.reg_read(UC_ARM64_REG_V25))
    print(">>> V29 = 0x%x"%mu.reg_read(UC_ARM64_REG_V29))

    # Dump where gcc /usually/ assembles global locations to
    for i in range(0x108d000, 0x108d160, 4):
        tmp = mu.mem_read(i, 4)
        print(">>> Read 4 bytes from [0x%x] = 0x" %(i), end="")
        for i in reversed(tmp):
            print("%x" %(i), end="")
        print("")
 
except Exception as e:
    print("ERROR: %s" % e) 



Emulate arm code
ERROR: name 'Uc' is not defined


## Parsing ARM64 Output For Unicorn

The unicorn code above takes binary intructions only, which need to be extracted from our generated code output. In order to test our generated code, it must first be ran with a aarch64-linux-gnu-{gcc,objdump} commands which produces output similar to what can be seen below:

All we care about is the binary output ("5280004b,b000046a,etc."), so we developed a simple parser to extract it and create a single line of binary code for Unicorn to take. Additionally the bytes are actually returned in reverse order from what Unicorn accepts, so the parser starts at the end of the number and returns each 4 byte sections in hex. This can be seen below, the parser reads in a file named 'out.dis' and prints out the hex code to standard out.

In [8]:
# opcode_scraper.py
# File designed for Python 2
# This file expects input similar to the above code cell.
# The first line should be main and any other
# GCC added code should be omitted.
import sys

def find_main(f):
	in_main = False
	for line in f:
		if len(line) > 3 and "00000000" not in line and '>:' not in line:
			get_ins_bytes(line)
		#elif in_main: break
		elif "<main>" in line:
			in_main = True

def get_ins_bytes(line):
	i = 0
	while line[i] != '\t': i += 1
	i += 1
	for j in range(8,0,-2):
		printHexByte(line[i+j-2:i+j])

def printHexByte(hex_byte):
	global code
	hb = ""
	hb += chr(int(hex_byte,16))
	code += hb
	

f = open('out.dis', 'r')
global code
code = ""
find_main(f)
sys.stdout.write(code)

FileNotFoundError: [Errno 2] No such file or directory: 'out.dis'

## Future Work

While we accomplished a lot during the duration of this project there is a plethora of future opportunities. First and foremost the ARMv8 backend could be cleaned up immensely to better fit with the ARM architecture. 

Secondly, proper calling conventions could be implemented for the architecture. Currently, similar to the original implementation of CGMips, all arguements are passed on the stack, however ARMv8 passes the first few through the w0+ registers.

Thirdly, ARMv8 supports SIMD instructions to perform operations between elements of a vector itself, such as summing the elements and placing it into a w/x register. This is an excellent opportunity to implement a builtin reduce method which could perform basic operations such as summing the elements of an array.

Fourthly, we would like to bring CGMips and CGAst into line with CGARMv8 by implementing SIMD equivalent blocks of code in each backend to truly allow the backends to be interchangable. 

Fifthly, we would like to modify the workings of Bool in P0 to allow for SIMD operations on Arrays of Bool within the ARMv8 backend. 

Finally, another future endeavour could be optimizing the emitted ARMv8 code to be more efficient than it currently is.

## References

(1) http://infocenter.arm.com/help/topic/com.arm.doc.ihi0055b/IHI0055B_aapcs64.pdf

    Procedure Calls in ARMv8

(2) https://www.element14.com/community/servlet/JiveServlet/previewBody/41836-102-1-229511/ARM.Reference_Manual.pdf
    
    Reference Manual for ARMv8 instruciton set

(3) https://quequero.org/2014/04/introduction-to-arm-architecture/
    
    Basic overview of ARM

(4) http://www.unicorn-engine.org/
    
    The Unicorn ARM virtual CPU emulator used to test output code
    
(5) https://w3challs.com/syscalls/?arch=arm_strong

    The Linux ARM syscall code table consulted for implementing syscalls.
    
(6) http://infocenter.arm.com/help/topic/com.arm.doc.den0024a/BABGBFBF.html
    http://infocenter.arm.com/help/topic/com.arm.doc.den0024a/CEGDJDJD.html
    
    Resources consulted to finally figure out the NEON_ENABLE instructions needed for Unicorn
    
(7) https://gcc.gnu.org/
    
    Assembler and disassembler used to turn code emitted from P0 into opcode format suitable for Unicron

